In [1]:
import math
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
# train and test data path
DATA_TRAIN_PATH = '../kaggle/Porto Seguro/input/train.csv'
DATA_TEST_PATH = '../kaggle/Porto Seguro/input/test.csv'

In [3]:
train = pd.read_csv(DATA_TRAIN_PATH)
test = pd.read_csv(DATA_TEST_PATH)
test.insert(1,'target',0)
print(train.shape)
print(test.shape)

(595212, 59)
(892816, 59)


In [4]:
x = pd.concat([train,test])
x = x.reset_index(drop=True)
unwanted = x.columns[x.columns.str.startswith('ps_calc_')]
x.drop(unwanted,inplace=True,axis=1)

In [5]:
features = x.columns[2:]
categories = []
for c in features:
    trainno = len(x.loc[:train.shape[0],c].unique())
    testno = len(x.loc[train.shape[0]:,c].unique())
    print(c,trainno,testno)

ps_ind_01 8 8
ps_ind_02_cat 5 5
ps_ind_03 12 12
ps_ind_04_cat 3 3
ps_ind_05_cat 8 8
ps_ind_06_bin 2 2
ps_ind_07_bin 2 2
ps_ind_08_bin 2 2
ps_ind_09_bin 2 2
ps_ind_10_bin 2 2
ps_ind_11_bin 2 2
ps_ind_12_bin 2 2
ps_ind_13_bin 2 2
ps_ind_14 5 5
ps_ind_15 14 14
ps_ind_16_bin 2 2
ps_ind_17_bin 2 2
ps_ind_18_bin 2 2
ps_reg_01 10 10
ps_reg_02 19 19
ps_reg_03 5013 5046
ps_car_01_cat 13 13
ps_car_02_cat 3 3
ps_car_03_cat 3 3
ps_car_04_cat 10 10
ps_car_05_cat 3 3
ps_car_06_cat 18 18
ps_car_07_cat 3 3
ps_car_08_cat 2 2
ps_car_09_cat 6 6
ps_car_10_cat 3 3
ps_car_11_cat 104 104
ps_car_11 5 5
ps_car_12 184 201
ps_car_13 70482 83769
ps_car_14 850 885
ps_car_15 15 15


In [6]:
x.loc[:,'ps_reg_03'] = pd.cut(x['ps_reg_03'], 50,labels=False)
x.loc[:,'ps_car_12'] = pd.cut(x['ps_car_12'], 50,labels=False)
x.loc[:,'ps_car_13'] = pd.cut(x['ps_car_13'], 50,labels=False)
x.loc[:,'ps_car_14'] =  pd.cut(x['ps_car_14'], 50,labels=False)
x.loc[:,'ps_car_15'] =  pd.cut(x['ps_car_15'], 50,labels=False)

In [7]:
test = x.loc[train.shape[0]:].copy()
train = x.loc[:train.shape[0]].copy()

In [8]:
#Always good to shuffle for SGD type optimizers
train = train.sample(frac=1).reset_index(drop=True)

In [9]:
train.drop('id',inplace=True,axis=1)
test.drop('id',inplace=True,axis=1)

In [10]:
train.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,0,0,2,0,0,-1,1,0,0,0,...,-1,1,2,1,103,3,29,4,39,37
1,0,1,2,7,1,0,0,1,0,0,...,1,1,2,1,1,0,31,9,41,46
2,0,1,1,2,0,0,1,0,0,0,...,1,1,2,1,44,3,31,6,42,32
3,0,0,1,6,0,0,0,0,0,1,...,1,0,2,1,1,0,31,10,43,48
4,0,4,1,9,0,0,0,0,0,1,...,1,1,0,1,57,2,30,5,42,37


In [11]:
target = train.target
train.drop('target', inplace=True, axis=1)
test.drop('target', inplace=True, axis=1)
n_train = train.shape[0]

train_test = pd.concat((train, test)).reset_index(drop=True)

In [12]:
train_test.ps_reg_01 = (train_test.ps_reg_01 * 10).astype('int')
train_test.ps_reg_02 = (train_test.ps_reg_02 * 10).astype('int')

In [13]:
dummy_df = pd.DataFrame()
for col in train_test.columns:
    dummy = pd.get_dummies(train_test[col].astype('category'))
    columns = dummy.columns.astype(str).tolist()
    columns = [col + '_' + w for w in columns]
    dummy.columns = columns
    dummy_df = pd.concat((dummy_df, dummy), axis=1)

In [14]:
dummy_df.head()

,ps_ind_01_0,ps_ind_01_1,ps_ind_01_2,ps_ind_01_3,ps_ind_01_4,ps_ind_01_5,ps_ind_01_6,ps_ind_01_7,ps_ind_02_cat_-1,ps_ind_02_cat_1,...,ps_car_15_29,ps_car_15_32,ps_car_15_35,ps_car_15_37,ps_car_15_40,ps_car_15_42,ps_car_15_44,ps_car_15_46,ps_car_15_48,ps_car_15_49
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
train = dummy_df.values[:n_train, :]
test = dummy_df.values[n_train:, :]
train.shape, test.shape

((595213, 408), (892816, 408))

In [16]:
from sklearn.cross_validation import train_test_split

C:\Users\zhaoy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [56]:
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=2)

In [57]:
y_train = y_train.reshape([-1, 1])
y_test = y_test.reshape([-1, 1])
y_train.shape, y_test.shape

((476170, 1), (119043, 1))

In [58]:
x_train.shape, x_test.shape

((476170, 408), (119043, 408))

In [59]:
import tensorflow as tf

In [60]:
n = x_train.shape[1]
# 潜在因子，越大拟合能力越强，越小泛化能力越强
k = 4

X = tf.placeholder('float', shape=[None, n])
y = tf.placeholder('float', shape=[None, 1])

# 初始化 0 次项，1 次项，辅助向量 v
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([n])) 
V = tf.Variable(tf.random_normal([n, k], stddev=0.01))

# FM 公式前两部分
linear_terms = tf.add(w0,
                      tf.reduce_sum(tf.multiply(W, X),
                                    axis=1,
                                    keep_dims=True))

# FM 公式最后一部分
interactions = (tf.multiply(0.5,
                tf.reduce_sum(
                    tf.subtract(
                        tf.pow(tf.matmul(X, V), 2),
                        tf.matmul(tf.pow(X, 2), tf.pow(V, 2))),
                    1, keep_dims=True)))

# 交叉熵损失
logits = tf.add(linear_terms, interactions)
y_hat = tf.sigmoid(logits)
loss = tf.losses.sigmoid_cross_entropy(y, logits)

eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.


In [61]:
N_EPOCHS = 10
batch_size = 512
num_steps = x_train.shape[0] // batch_size + 1
batch_size, num_steps

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        indices = np.arange(x_train.shape[0])
        np.random.shuffle(indices)
        x_train, y_train = x_train[indices], y_train[indices]
        for step in range(num_steps):
            offset = (step * batch_size) % (x_train.shape[0] - batch_size)
            x_batch = x_train[offset:(offset + batch_size), :]
            y_batch = y_train[offset:(offset + batch_size), :]
            sess.run(optimizer, feed_dict={X: x_batch, y: y_batch})
        test_pred = sess.run(y_hat, feed_dict={X: x_test}).flatten()
        auc = metrics.roc_auc_score(y_score=test_pred, y_true=y_test.flatten())
        logloss = metrics.log_loss(y_pred=test_pred.tolist(), y_true=y_test.flatten().tolist())
        print(2*auc-1, logloss)
    v = sess.run(V, feed_dict={X: x_test})
    w = sess.run(W, feed_dict={X: x_test})
    w0 = sess.run(w0, feed_dict={X: x_test})

0.276329349622 0.151436589682
0.282447552492 0.150675390616
0.285323364728 0.150542314571
0.286699193082 0.150752264632
0.288834971524 0.150473857297
0.289183018366 0.150369272507
0.287936809376 0.150305516604
0.291049830386 0.1502848509
0.29182346341 0.150220800439
0.291238518446 0.150326201981


In [62]:
v.shape

(408, 4)

In [63]:
v_df = pd.DataFrame(v)

In [64]:
v_df.to_csv('fm_embeddings.csv', index=False)